## 설치 준비 데이터로드

In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 10.1MB/s 
     |████████████████████████████████| 1.1MB 41.0MB/s 
     |████████████████████████████████| 2.9MB 50.6MB/s 
     |████████████████████████████████| 890kB 42.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=52f7ce5400ba64213c56420f707660b071df36ed210255a2f7e413ae39def469
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [4]:
# 네이버 영화리뷰 감정분석 데이터
!git clone https://github.com/e9t/nsmc.git

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [5]:
# 훈련셋 150000개 테스트셋 50000개
train = pd.read_csv("nsmc/ratings_train.txt", sep="\t")
test = pd.read_csv("nsmc/ratings_test.txt", sep="\t")

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


In [6]:
# id 회원정보 / document 리뷰 문장 / label 긍정1 부정0
train.sample(10)

,id,document,label
149203,8540108,영화 운동하는 캐릭터 중에 록키가 제일 착해서 맘에 드네유.,1
108805,5367134,정말 최고였다,1
134165,8679096,"옛날 그때는 국민학교 로 부르던 시절 ,, 영화 속 학교 풍경 책,걸상 책가방 학생...",1
99636,9558427,'하이틴 로맨스 판타지 SF'는 이제 금물로 했으면.,0
27407,7936437,왜 평점 0점은 없죠? 감독은 왜 한국인이 보는 영화에 한국인을 이렇게 비하 하는 ...,0
86733,9652213,성지순례하러 왔습니다,0
29186,10244027,여태 본 영화중에 최최악영화.. 웃긴요소는 많이 넣은 것 같은데 하나도 안 웃기고 ...,0
35840,10110252,이렇게 괜찮은영화가 왜 평점이이러지...먹먹하지만 좋은영화라고 생각한다,1
112126,8206840,정말지루하고재미없고잠오는영화///,0
69219,9556018,그닥 재미 없음....,0


## 훈련셋 전처리

In [7]:
#리뷰 문장 추출
sentences = train['document']
sentences[:10]

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
5        막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
6                                원작의 긴장감을 제대로 살려내지못했다.
7    별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...
8                               액션이 없는데도 재미 있는 몇안되는 영화
9        왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?
Name: document, dtype: object

In [8]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]',
 '[CLS] 막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움. [SEP]',
 '[CLS] 원작의 긴장감을 제대로 살려내지못했다. [SEP]',
 '[CLS] 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네 [SEP]',
 '[CLS] 액션이 없는데도 재미 있는 몇안되는 영화 [SEP]',
 '[CLS] 왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나? [SEP]']

BERT는 Classification을 뜻하는 [CLS] 심볼이 문장의 가장 앞에 삽입된다. 파인튜닝시 출력에서 이 위치의 값을 사용하여 분류를 한다. [SEP]는 Seperation으로 두 문장을 구분하는 역할을 한다.

In [9]:
#라벨 추출
labels = train['label'].values
labels

array([0, 1, 0, ..., 0, 1, 0])

In [10]:
#BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print(sentences[0])
print(tokenized_texts[0])


[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]
['[CLS]', '아', '더', '##빙', '.', '.', '진', '##짜', '짜', '##증', '##나', '##네', '##요', '목', '##소', '##리', '[SEP]']


BERT는 형태소분석으로 토큰을 분리하지 않고 WordPiece라는 통계적인 방식을 사용한다. 한 단어 내에서 자주 나오는 글자들을 붙여 하나의 토큰으로 만든다. 이렇게하면 언어에 상관없이 토큰을 생성할 수 있고, 신조어같이 사전에 없는 단어들을 처리하기에 좋다.
\##은 앞 토큰과 이어진다는 표시이다. 토크나이저는 여러 언어의 데이터를 기반으로 만든 'bert-base-multilingual-cased'를 사용한다.

In [11]:
#입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

#토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

#문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9519,   9074, 119005,    119,    119,   9708, 119235,
         9715, 119230,  16439,  77884,  48549,   9284,  22333,  12692,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

딥러닝 모델에 토큰 자체를 입력으로 넣을 수 없다. 임베딩 레이어에는 토큰을 숫자로 된 인덱스로 변환하여 사용한다. BERT의 토크나이저는 {단어토큰:인덱스}로 구성된 단어사전을 가지고 있다. 이를 참조하여 토큰을 인덱스로 바꾸어준다.

In [12]:
#어텐션 마스크 초기화
attention_masks=[]

#어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
#패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
  seq_mask=[float(i>0) for i in seq]
  attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [13]:
#훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels  = train_test_split(input_ids, labels, random_state=2018, test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2018, test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9711,  11489,   9364,  41850,   9004,  32537,   9491,  35506,
         17360,  48549,    119,    119,   9477,  26444,  12692,   9665,  21789,
         11287,   9708, 119235,   9659,  22458, 119136,  12965,  48549,    119,
           119,   9532,  22879,   9685,  16985,  14523,  48549,    119,    119,
          9596, 118728,    119,    119,   9178, 106065, 118916,    119,    119,
          8903,  11664,  11513,   9960,  14423,  25503, 118671,  48549,    119,
           119,  21890,   9546,  37819,  22879,   9356,  14867,   9715, 119230,
        118716,  48345,    119,   9663,  23321,  10954,   9638,  35506, 106320,
         10739,  20173,   9359,  19105,  11102,  42428,  17196,  48549,    119,
           119,    100,    117,   9947,  12945,   9532,  25503,   8932,  14423,
         35506, 119050,  11903,  14867,  10003,  14863,  33188,  48345,    119,
           102,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [14]:
#배치 사이즈
batch_size = 32

#파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
#학습시 배치 사이즈만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## 테스트셋 전처리

In [15]:
#리뷰 문장 추출
sentences = test['document']
sentences[:10]

0                                                  굳 ㅋ
1                                 GDNTOPCLASSINTHECLUB
2               뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3                     지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4    3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
5                                   음악이 주가 된, 최고의 음악영화
6                                              진정한 쓰레기
7             마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다
8    갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...
9       이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..
Name: document, dtype: object

In [16]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 굳 ㅋ [SEP]',
 '[CLS] GDNTOPCLASSINTHECLUB [SEP]',
 '[CLS] 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 [SEP]',
 '[CLS] 지루하지는 않은데 완전 막장임... 돈주고 보기에는.... [SEP]',
 '[CLS] 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? [SEP]',
 '[CLS] 음악이 주가 된, 최고의 음악영화 [SEP]',
 '[CLS] 진정한 쓰레기 [SEP]',
 '[CLS] 마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 [SEP]',
 '[CLS] 갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 [SEP]',
 '[CLS] 이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.. [SEP]']

In [17]:
#라벨 추출
labels = test['label'].values
labels

array([1, 0, 0, ..., 0, 0, 0])

In [18]:
#BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print(sentences[0])
print(tokenized_texts[0])

[CLS] 굳 ㅋ [SEP]
['[CLS]', '굳', '[UNK]', '[SEP]']


In [19]:
#입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

#토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

#문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([ 101, 8911,  100,  102,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [20]:
#어텐션 마스크 초기화
attention_masks=[]

#어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
#패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
  seq_mask=[float(i>0) for i in seq]
  attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [21]:
#데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([ 101, 8911,  100,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
tensor(1)
tensor([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [22]:
#배치 사이즈
batch_size = 32

#파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
#학습시 배치 사이즈만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

##모델 생성

In [23]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("We will use the GPU:", torch.cuda.get_device_name(0))

We will use the GPU: Tesla T4


In [24]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [29]:
#옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8) #lr 학습률, eps 0으로 나누는 것을 방지하기 위한 epsilon값

#에폭수
epochs = 3

#총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

#학습률을 조금씩 감소시키는 스케쥴러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

## 모델 학습

In [32]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [27]:
#시간 표시 함수
def format_time(elapsed):
  elapsed_rounded = int(round((elapsed)))
  return str(datetime.timedelta(seconds=elapsed_rounded))

In [33]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:05:46.
  Batch 1,000  of  4,219.    Elapsed: 0:11:32.
  Batch 1,500  of  4,219.    Elapsed: 0:17:18.
  Batch 2,000  of  4,219.    Elapsed: 0:23:03.
  Batch 2,500  of  4,219.    Elapsed: 0:28:48.
  Batch 3,000  of  4,219.    Elapsed: 0:34:34.
  Batch 3,500  of  4,219.    Elapsed: 0:40:19.
  Batch 4,000  of  4,219.    Elapsed: 0:46:04.

  Average training loss: 0.08
  Training epcoh took: 0:48:35

Running Validation...
  Accuracy: 0.86
  Validation took: 0:01:49

======== Epoch 2 / 3 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:05:46.
  Batch 1,000  of  4,219.    Elapsed: 0:11:32.
  Batch 1,500  of  4,219.    Elapsed: 0:17:18.
  Batch 2,000  of  4,219.    Elapsed: 0:23:04.
  Batch 2,500  of  4,219.    Elapsed: 0:28:50.
  Batch 3,000  of  4,219.    Elapsed: 0:34:37.
  Batch 3,500  of  4,219.    Elapsed: 0:40:23.
  Batch 4,000  of  4,219.    Elapsed: 0:46:09.

  Average training loss: 0.11
  

## 테스트셋 평가

In [35]:
#시작 시간 설정
t0 = time.time()

#평가모드로 변경
model.eval()

#변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

#데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
  #경과 정보 표시
  if step % 100 == 0 and not step == 0:
    elapsed = format_time(time.time() - t0)
    print(" Batch{:>5,} of {:>5,}. Elapsed: {:}.".format(step, len(test_dataloader), elapsed))

    #배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)

    #배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch

    #그래디언트 계산 안함
    with torch.no_grad():
      #Forward 수행
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    #로스 구함
    logits = outputs[0]

    #CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    #출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print(" Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print(" Validation took: {:}".format(format_time(time.time() - t0)))


 Batch  100 of 1,563. Elapsed: 0:00:00.
 Batch  200 of 1,563. Elapsed: 0:00:00.
 Batch  300 of 1,563. Elapsed: 0:00:01.
 Batch  400 of 1,563. Elapsed: 0:00:01.
 Batch  500 of 1,563. Elapsed: 0:00:01.
 Batch  600 of 1,563. Elapsed: 0:00:01.
 Batch  700 of 1,563. Elapsed: 0:00:02.
 Batch  800 of 1,563. Elapsed: 0:00:02.
 Batch  900 of 1,563. Elapsed: 0:00:02.
 Batch1,000 of 1,563. Elapsed: 0:00:02.
 Batch1,100 of 1,563. Elapsed: 0:00:03.
 Batch1,200 of 1,563. Elapsed: 0:00:03.
 Batch1,300 of 1,563. Elapsed: 0:00:03.
 Batch1,400 of 1,563. Elapsed: 0:00:03.
 Batch1,500 of 1,563. Elapsed: 0:00:04.

 Accuracy: 0.89
 Validation took: 0:00:04


Utilize to HMG

In [36]:
!ls

matge.csv  nsmc  sample_data


In [37]:
import pandas as pd
import numpy as np

df_raw = pd.read_csv("matge.csv", encoding="cp949")

In [38]:
df_raw.info

<bound method DataFrame.info of           식당명                                              리뷰 문장
0     레스토랑 아진                        연남동쪽에 있고 파스타 1.5 부채살 스테끼3.4
1     레스토랑 아진                 와인까지 세트로 5.4 저 감자 뭐시기는 0.5 레몬맛+소금맛
2     레스토랑 아진  알리오올리오인데 우리가 흔히 먹는 맛은 아니고 뭔가 치즈맛인가 맛표현 어려운데 맛있...
3     레스토랑 아진  스테끼는 뭐 당근퓨레랑 저 풀때기 뭔지 모르는데 같이 샥 해먹으면 아주 부드러움. ...
4     레스토랑 아진  남친이랑 100일 기념으로 갔는데 예약을 많이 하고 가는 듯. 예약 선입금 2만원 ...
...       ...                                                ...
5304      NaN                                                NaN
5305      NaN                                          재방문 의사 있음
5306      NaN                                                NaN
5307      NaN                 (기본토스트보다는 돈 조금 보태서 햄이나 베이컨들어간걸 추천)
5308      NaN                                               ]"["

[5309 rows x 2 columns]>

In [39]:
df_raw.sample(10)

,식당명,리뷰 문장
5023,당인동 국수공장,ㅡ넒음
1001,정문 앞 롤링파스타,다른 지점보다 가지라던지 재료가 조금 덜 들어있었지만
3692,홍아지트,NaN
1964,플라워,그리고 화장실이 깨끗하다!! &lt;- 중요
2110,베리메리,NaN
3941,NaN,NaN
1319,NaN,"]""["""
4940,오키나와루,김부각에 와사비를 섞었는데 달아진 맛
1921,시즌샌드위치,진짜 맛있었음
2987,합정 소문,깔끔하니 맛있음! 부드럽고 소스 맛있음. 긁어먹음.


In [40]:
df_raw.isnull().sum()

식당명      1048
리뷰 문장    1381
dtype: int64

In [41]:
df_dropped = df_raw.dropna(axis=0)

In [42]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3298 entries, 0 to 5291
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   식당명     3298 non-null   object
 1   리뷰 문장   3298 non-null   object
dtypes: object(2)
memory usage: 77.3+ KB


In [43]:
df_dropped.sample(10)

,식당명,리뷰 문장
2732,다디치 커피,맨날 사람 없어서 사라질까봐 올림... 아저씨 아주머니 엄청 착하시고 친절하심 그리...
4183,힙정옥,"곰탕하면 '하동관'이라는 노포를 떠올리게 되는데, 저는 이제 합정옥을 더 많이 가게..."
290,연남물갈비,저 야채 속에 등갈비 있는데 연해서 맛있더라
1349,호호미역 홍대점,사장님 들어갈때 반갑게 맞아주시고
3060,신선마라탕,빽다방 골목쪽에 있는 신선 마라탕 지금 50% 할인합니다!! 많이 와주세요ㅠㅠㅠ 원...
1356,호호미역 홍대점,미리 볶아둔게 아니라 주문들어가고 나서 볶기 시작하셨음. 천원짜리 반찬 포장도 따로...
3460,퍼센테지,퍼센테지 오레오스무디 먹으러갔는데 카페 임대하더라ㅜㅜ
2385,상수 비스트로보이,연어로제파스타 8500 / 불고기뚝배기파스타 8900
3588,고토히라 우동,메뉴: 키노코 우동
3397,연남동 감칠,분위기도 좋고 가성비도 갠츈갠츈!!


In [44]:
df_dropped['리뷰 문장'] = df_dropped['리뷰 문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣|0-9 ]","")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
df_dropped.sample(10)

,식당명,리뷰 문장
4166,홍대 수카츠,직원들은 친절했음
182,망원 멘지,최고의 맛
3379,북촌손만두,오늘 북촌손만두에서 육개장 먹었는데요 이 오른쪽 무 한조각이 나와서 사장님은 국물낼...
3362,홍대 우산,홍대 7출 앞쪽 도보 2분 이층 버거집 올드루키스버거
1823,더피자보이즈,그래도 피자먹고싶을때 또 갈거임
1101,요요스시,합정역 5번 출구 근처에 있는 요요스시임
2806,북창동순두부,저번에도 그 아주머니 불친절하셔서 좀 짜증났었는데 덕분에 갈일이 더블로 없어졌네용
15,칸다소바,마제소바 9500원
1070,짱주먹밥,날치알 3000원 계란후라이 800원
3891,춘리 슈가 흑당 버블티,춘리 슈가 흑당 버블티 새로 오픈했는데 오픈할인해서 4700원 비싸긴한데


In [47]:
def convert_input_data(sentences):

  #BERT의 토크나이저로 문장을 토큰으로 분리
  tokenized_texts = [tokenizer.tokenize(sen) for sen in sentences]

  #입력 토큰의 최대 시퀀스 길이
  MAX_LEN = 128

  #토큰을 숫자 인덱스로 변환
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  #문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  #어텐션 마스크 초기화
  attention_masks = []

  #어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
  #패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
  for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

  #데이터를 파이토치의 텐서로 변환
  inputs = torch.tensor(input_ids)
  masks = torch.tensor(attention_masks)

  return inputs, masks

In [48]:
def test_sentences(sentences):
  
  #평가모드로 변경
  model.eval()

  #문장을 입력 데이터로 변환
  inputs, masks = convert_input_data(sentences)

  #데이터를 GPU에 넣음
  b_input_ids = inputs.to(device)
  b_input_mask = masks.to(device)

  #그래디언트 계산 안함
  with torch.no_grad():
    #Forward 수행
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  #로스 구함
  logits = outputs[0]

  #CPU로 데이터 이동
  logits = logits.detach().cpu().numpy()

  return logits

In [49]:
logits = test_sentences(["연기는 별로지만 재미 하나는 끝내줌!"])

print(logits)
print(np.argmax(logits))

[[-3.047224   3.0989723]]
1


In [50]:
logits = test_sentences(["이딴게 영화냐 ㅉㅉ"])

print(logits)
print(np.argmax(logits))

[[ 3.3799307 -3.0779366]]
0


In [51]:
#0은 부정 1은 긍정
def good_or_bad(sentence):
  logits = test_sentences([sentence])
  determine = np.argmax(logits)
  
  return determine

In [52]:
#극찬 영화평 BERT로 확인해보기
good_or_bad("와 개쩐다 정말 세계관 최강자들의 영화다")

1

In [53]:
#리뷰 라벨링
def label_review_bert(series_object):
  list_scores=[]
  list_reviews = series_object.to_list()
  for item in list_reviews:
    score = good_or_bad(item)
    list_scores.append(score)
  return list_scores

In [54]:
df_dropped["BERT_SCORE"] = label_review_bert(df_dropped["리뷰 문장"]) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [55]:
df_dropped.sample(10)

,식당명,리뷰 문장,BERT_SCORE
1569,마코토,더밥때부터 종종 가던 가게인데 마코토로 옮기고도 연어덮밥 땡길때 갑니다 ㅎㅎ,1
2203,진만두,산울림 소극장 삼거리 안쪽에 위치한 만두가게 전통 중국식 만두를 판다 가성비는 다른...,1
4537,더리터,먹는것을추천함,1
4696,홍커피,홍커피 딸바 2500,0
2685,보물섬식당,오늘 점심은 돈까스각이었다,0
3667,친구네집빈날,사진 속 칵테일은 미도리샤워 칵테일도 이뻐서 추천,1
3010,상수 미쁘다,존나 맛있다,1
3748,짬뽕지존,ㅡ여기서 짬뽕을먹어도되나 싶은 궁전,1
299,연남물갈비,글구 일회용 앞치마도 부족했어 뭔가 재고관리 능력이 안타깝더라,0
123,고피자 상수점,맛없는데 파스타 두개에 5500원이라 저렴한 맛에는 먹을만 함,0


In [56]:
df_dropped.to_csv(f"HMG_Sentiment_Analysis_BERT.csv",encoding="cp949")